In [2]:
import sys
import json
sys.path.append('..')
from src.analysis.model_performances import *
from src.questions_construction.questions import *
import pandas as pd
EVERYTHING_POSTFIX = 'everything'

In [3]:
questions_dir = f'{DATA_PATH}/questions_m1'
questions_by_id = gather_questions(questions_dir)
# sanity_checks()
data_all, missing_data = gather_data(questions_by_id)

data is gathered


In [26]:
class TrueFalseStatsCustom(TrueFalseStats):
    def __init__(self, filtered_data, plan_length, question_category, ramifications, model_name, prompt_type, domain,
                 substitutions, score_type=F1_SCORE_KEY):
        super().__init__(filtered_data, plan_length, question_category, ramifications, model_name, prompt_type, domain,
                 substitutions, score_type=F1_SCORE_KEY)
        self.answer_type = TRUE_FALSE_ANSWER_TYPE
        self.score_type = score_type
        self.data = filtered_data

def filter_multi_selector_modified(data_all, ramifications, model_name, prompt_type, answer_type, substitutions, plan_length, other_keys_ls):
    """ if ALL_DOMAINS_KEY or ALL_CATEGORIES_KEY or ALL_LENGTHS_KEY selects multiple values from data_all"""
    filter_by = base_filter(ramifications, model_name, prompt_type, answer_type, substitutions)
    filter_by.append((OUT_OBJ_PLAN_LENGTH, {plan_length}))
    filter_by.extend(other_keys_ls)
    
    results = []
    for d in data_all:
        if all(d[k] in v for k, v in filter_by):
            results.append(d)  # dedup if there was a screw up in the evaluations stage and things were computed >1 times
    return results

In [27]:
model_name = 'gemini'#'gemma-2b-it'
prompt_type = 'few_shot_1'
question_category = ALL_QUESTION_CATEGORIES_KEY
substitutions = WITHOUT_RANDOM_SUB
ramifications = WITHOUT_RAMIFICATIONS
answer_type = TRUE_FALSE_ANSWER_TYPE

plan_length = 1
domain = ALL_DOMAINS_KEY

In [35]:
by_fluent_type = {}
for fluent_type in list(FLUENT_TYPES_LIST) + [None]:
    data = filter_multi_selector_modified(data_all, ramifications, model_name, prompt_type, answer_type, substitutions, plan_length, [(OUT_OBJ_FLUENT_TYPE, {fluent_type})])
    stats = TrueFalseStatsCustom(data, plan_length, question_category, ramifications, model_name, prompt_type, domain, substitutions)
    by_fluent_type[fluent_type] = stats.compute()['result']

In [36]:
by_fluent_type

{'base_fluents': 0.531055900621118,
 'derived_fluents': 0.4934169278996865,
 'persistent_fluents': 0.5070539419087137,
 'static_fluents': 0.4739795918367347,
 None: 0.47448053258018963}

# By positive and neg fluents

In [33]:
by_is_pos_fluent_question = {}
for is_pos_fluent_question in [True, False, None]:
    data = filter_multi_selector_modified(data_all, ramifications, model_name, prompt_type, answer_type, substitutions, plan_length, [(OUT_OBJ_IS_POS_FLUENT_QUESTION, {is_pos_fluent_question})])
    stats = TrueFalseStatsCustom(data, plan_length, question_category, ramifications, model_name, prompt_type, domain, substitutions)
    by_is_pos_fluent_question[is_pos_fluent_question] = stats.compute()['result']

{True: 0.4911489066296425, False: 0.605080831408776, None: 0.4621561142556272}

In [34]:
by_is_pos_fluent_question

{True: 0.4911489066296425, False: 0.605080831408776, None: 0.4621561142556272}

In [ ]:
def to_df(results_all, model_name, answer_type, prompt_type, subs, domains):
    d_ramifications_pretty = {
        WITH_RAMIFICATIONS : 'R',
        WITHOUT_RAMIFICATIONS : 'No R'
    }
    d_answer_type_pretty = {
        TRUE_FALSE_ANSWER_TYPE : 'T/F',
        FREE_ANSWER_TYPE : 'Free'
    }
    
    index = []
    data = []
    for plan_length in PLAN_LENGTHS:
        for ramifications in RAMIFICATION_TYPES:
            index.append((plan_length, d_ramifications_pretty[ramifications], d_answer_type_pretty[answer_type]))
            data_columns = {}
            for domain in domains:
                by_fluent_type = {}
                for fluent_type in list(FLUENT_TYPES_LIST) + [None]:
                    data = filter_multi_selector_modified(data_all, ramifications, model_name, prompt_type, answer_type, substitutions, plan_length, [(OUT_OBJ_FLUENT_TYPE, {fluent_type})])
                    stats = TrueFalseStatsCustom(data, plan_length, question_category, ramifications, model_name, prompt_type, domain, substitutions)
                    by_fluent_type[fluent_type] = stats.compute()['result']
                    
                
                data_columns[domain] = filter_single_selector(results_all, plan_length, question_category, ramifications, model_name, prompt_type, domain, answer_type, subs)
            data.append(data_columns)
    return pd.DataFrame(data, index = index)